# Title
Back to basics! Let's revisit basic ML algorithms from scratch. In this notebook, I implement a vanilla version of decision tree classifier from scratch using pandas and numpy. In addition, also implemented ROC Curve from scratch and ended with comparing both decision tree and roc functions with SKlearn's implementation on breast cancer dataset from Sklearn. 

## Libraries for basic operations

In [ ]:
import numpy as np
from abc import ABC
import pandas as pd 
from collections import defaultdict

## Vanilla Decision Tree Classifier Class

Some points to note:
- Evaluation metric used to create splits: Weighted gini impurity. Recommended reading: https://www.learndatasci.com/glossary/gini-impurity/
- Regularization methods: max_depth (maximum depth of decision tree) and min_leaf_size (minimum samples in each leaf)

In [ ]:
class VanillaDecisionTreeClassifier(ABC):
    def __init__(self, target='y', max_depth=3, min_leaf_size=1):
        self.max_depth = max_depth
        self.min_leaf_size = min_leaf_size 
        self.target = target # name of target feature. single dataframe with feature and target is passed during fit
        self.lowest_impurity = 1e3 # initialization of high number
    
    def total_split_impurity(self, left_df, right_df):
        """
        Measurement used to find best feature-value pair for each split of classification decision trees.  
        
        This function gives total gini impurity value (scalar) given data that goes in left and right nodes
        
        left_df: (pd.DataFrame) Data going to left node after split
        right_df: (pd.DataFrame) Data going to right node after split
        
        Return: total gini impurity by node split weighted by samples going in left and right nodes
        """
        gini_left = self.gini_impurity(left_df)
        gini_right = self.gini_impurity(right_df)
        total_impurity = gini_left*(left_df.shape[0]/(left_df.shape[0] + right_df.shape[0])) + \
        gini_right*(right_df.shape[0]/(left_df.shape[0] + right_df.shape[0]))
        return total_impurity
    
    def gini_impurity(self, df):
        """
        Gini impurity of single node. For more info: https://www.learndatasci.com/glossary/gini-impurity/
        Ranges betwee 0 to 0.5 where 0 means most pure node 
        and a great candidate for split
        
        df: (pd.DataFrame) Data in a given node
        
        Return: gini impurity
        """
        ypos = (df[self.target] == 1).sum()
        yneg = (df[self.target] == 0).sum()
        ppos = ypos/(ypos+yneg)
        pneg = 1-ppos
        return 1 - ((ppos**2) + (pneg**2))
        
                          
    def find_best_split(self, df, current_depth=0):
        """
        Given data in current node (df), it finds feature name and it's value that would give best possible split.
        Uses total split impurity as measurement of best split. i.e. total split impurity should be lowest among all options
        This function is called recursively starting from root to leaf until stopping criteria is reached.
        
        df: (pd.DataFrame) Data in current node that we are trying to split
        current_depth: (int) Current depth of tree. 
                             Required to track the depth in order to stop when max depth is reached
                             
        Return: tree_dict (dict) Nested dictionary containing tree node attributes from root to leaves  
        """
        # only split if the new nodes give lower impurity than what we have in current node
        self.lowest_impurity = self.gini_impurity(df) 
        
        # Check all features to find feature value pair giving that would give best split 
        for feat_name in df.drop(self.target,axis=1).columns:  
            self.find_best_split_single_feat(df, feat_name)
        
        # if no better split found after all search, just return the y_mean value at this node
        if self.lowest_impurity >= self.gini_impurity(df) or current_depth > self.max_depth: 
            return df[self.target].mean()
        
        best_feat_name, y_bucket, best_feat_val = self.best_feat_name, self.y_bucket, self.best_feat_val
        # tree dict is used to save nested tree structure that will later be used to predict
        tree_dict = defaultdict() 
        tree_dict[best_feat_name]  = {}
        tree_dict[best_feat_name]['feat_value'] = best_feat_val
        tree_dict[best_feat_name]['ymean'] = y_bucket
        
        # simply sort by selected feature name and take left-right
        df_sort = df.copy().sort_values(by=self.best_feat_name, ascending=True)  # Not optimal to sort each time
        left_tree_df = df_sort[df_sort[self.best_feat_name] <= self.best_feat_val]
        right_tree_df = df_sort[df_sort[self.best_feat_name] > self.best_feat_val]

        if (len(left_tree_df)==0) or (len(right_tree_df)==0):  # Creating balanced tree (optional)
            return
                
        # Go down the tree if it finds better splits. If it does, create nested dicts and replace value at nodes
        current_depth+=1
        tree_dict[best_feat_name]['left'] = self.find_best_split(left_tree_df, current_depth)
        tree_dict[best_feat_name]['right'] = self.find_best_split(right_tree_df, current_depth)
        
        return tree_dict
    
    def find_best_split_single_feat(self, df, feat_name):
        
        """
        Given data in current node (df) and feature name to check, find the value that gives best possible 
        node split (of df) at this feature. Finding a feature value that gives lower impurity than 
        currently found impurity is not guaranteed. It will only select feature-value pair if it reduces the impurity.
        
        df: (pd.DataFrame) Data in current node we are trying to split
        feat_name: (str) Feature name we are trying to split on
        
        Return: None. Updates characterstics of best split value in class object. 
        """
        
        # It is not optimal to sort and reset index each time
        # But we are not really optimizing for now
        # This is a simple vanilla implementation for understanding
        df_sort= df.copy().sort_values(by=feat_name)
        df_sort.reset_index(drop=True, inplace=True)
        uniq_vals = df_sort[feat_name].unique()  # only check unique values to save some redundancy
        for i in uniq_vals: #range(self.min_leaf_size, df_sort.shape[0] - self.min_leaf_size): 
            
            # Left and Right datasets should be equal or larger than min_leaf_size. 
            left_df = df_sort[df_sort[feat_name] <= i] 
            right_df = df_sort[df_sort[feat_name] > i]
            
            if len(left_df)<self.min_leaf_size or len(right_df)<self.min_leaf_size:
                continue # skip this value if it gives a split node smaller than min accepted leaf size

            new_impurity = self.total_split_impurity(left_df, right_df)

            if new_impurity < self.lowest_impurity:
                # Update best split node attributes if new impurity is lower than lowest sofar
                self.lowest_impurity = new_impurity
                self.best_feat_name = feat_name
                self.best_feat_val = i
                self.y_bucket = df_sort[self.target].mean()

        
    def fit(self, df):
        """
        Calls find_best_split function and fits the tree with input dataframe
        
        df: (pd.DataFrame) Training data to fit the decision tree classfier
        
        Returns: (dict) Fitted tree 
        """
        self.fitted_tree = self.find_best_split(df)
        return self.fitted_tree
        
#     @property
#     def better_split_not_found(self): return self.lowest_impurity == 1e3  # impurity is not reduced 
    
    @staticmethod
    def _is_leaf_node(subtree):
        """
        subtree: (dict) Given a branch of tree, check whether it is leaf
        
        Return (bool) True if given tree branch is leaf node, False otherwise
        """
        if not subtree['left'] and not subtree['right']:  # if leaf, would not have any children
            return True  
        else: return False
    
    @staticmethod
    def _predict_row(fitted_tree, row):
        """
        fitted_tree: (dict) Output of self.fit
        row: (pd.DataFrame) Single row of test dataframe
        
        Return (Float) Prediction value for given input row. Between 0 to 1
        
        From top to bottom, traverse the whole tree and find which node does this data point lies in
        """
        
        if type(fitted_tree) == float:
            return fitted_tree # leaf has no further children
        
        key = list(fitted_tree.keys())[0] # feature index of current node
        fitted_key = fitted_tree[key]
        
        if VanillaDecisionTreeClassifier._is_leaf_node(fitted_key):
            return fitted_key['ymean']
        
        if fitted_key['feat_value'] >= row[key]:
            return VanillaDecisionTreeClassifier._predict_row(fitted_key['left'], row)
        else:
            return VanillaDecisionTreeClassifier._predict_row(fitted_key['right'], row)
        

        
    def predict(self, X_test):
        """
        Runs predict_row for each row of test dataframe
        
        fitted_tree: (dict) Output of self.fit
        X_test: (pd.DataFrame) Dataframe containing test data you want to predict using fitted decision tree
        
        Return: (List) Predictions from tree in given test data
        """
        
        preds = [VanillaDecisionTreeClassifier._predict_row(self.fitted_tree, X_test.iloc[i,:])\
                 for i in range(X_test.shape[0])]
        return preds

### Toy Data 1

In [ ]:
toy_data = pd.DataFrame({"x1": [1, 10, 432, 1, 60],
                         "x2":[2, 5, 10, 6, 6],
                         "x3":[10, 10, 10, 10, 10],
                         "y":[1, 1, 0, 0, 0],
                        })
toy_data

In [ ]:
dt = VanillaDecisionTreeClassifier()

In [ ]:
fitted_dt = dt.fit(toy_data)
fitted_dt

In [ ]:
dt.predict(toy_data)

## Toy Data 2 (sklearn breast cancer dataset)

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
sklearn_data = load_breast_cancer()
X, y = sklearn_data.data, sklearn_data.target
sklearn_datadf = pd.DataFrame(X)
sklearn_datadf['y'] = y

In [ ]:
# Train test split, random 75-25% splits

sklearn_datadf_train = sklearn_datadf.sample(frac=0.75, random_state=10)

test_idx = [i for i in range(len(sklearn_datadf)) if i not in sklearn_datadf_train.index]
sklearn_datadf_test = sklearn_datadf.iloc[test_idx, :]
sklearn_datadf_test.reset_index(drop=True, inplace=True)

In [ ]:
dt = VanillaDecisionTreeClassifier(max_depth=4, min_leaf_size=2)
fitted_dt = dt.fit(sklearn_datadf_train)

In [ ]:
fitted_dt

In [ ]:
test_preds = dt.predict(sklearn_datadf_test)
train_preds = dt.predict(sklearn_datadf_train)

## ROC curve from scratch

In [ ]:
def roc_curve_scratch(y_labels, y_scores):
    thresholds = np.linspace(0, 1, 1001)
    fprs, tprs = [], []
    for thres in thresholds:
        tp, tn, fp, fn = confusion_matrix(y_labels, y_scores, thres)
        tprs.append(tpr(tp, tn, fp, fn))
        fprs.append(fpr(tp, tn, fp, fn))
        
    # force first and last fpr, tpr at 0 and 1 thresholds
    fprs[0] = 1
    fprs[-1] = 0
    tprs[0] = 1
    tprs[-1] = 0
    return fprs, tprs, thresholds
    
def confusion_matrix(y_labels, y_scores, thres):
    y_preds = (y_scores >= thres).astype(int)
    tp = (np.equal(y_labels, 1) & np.equal(y_preds, 1)).sum()
    tn = (np.equal(y_labels, 0) & np.equal(y_preds, 0)).sum()
    fp = (np.equal(y_labels, 0) & np.equal(y_preds, 1)).sum()
    fn = (np.equal(y_labels, 1) & np.equal(y_preds, 0)).sum()
    
    return tp, tn, fp, fn

def tpr(tp, tn, fp, fn):
    return tp/(tp+fn)

def fpr(tp, tn, fp, fn):
    return fp/(fp+tn)

def auc_scratch(fprs, tprs):
    """
    Cut in small rectangles and sum areas
    """
    total_auc = 0.
    for i in range(1000):  # divide curve in 1000 rectangles
        total_auc += (fprs[i] - fprs[i+1])*((tprs[i+1] + tprs[i])/2.)
    return total_auc

In [ ]:
fprs, tprs, thresholds = roc_curve_scratch(sklearn_datadf_test['y'], np.array(test_preds))

In [ ]:
auc_scratch(fprs, tprs)

## Comparing with sklearn implementations of DecisionTreeClassifier and roc_curve

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

In [ ]:
dtc = DecisionTreeClassifier(max_depth=4, min_samples_leaf=2)
dtc.fit(sklearn_datadf_train.drop('y', axis=1), sklearn_datadf_train['y'])

In [ ]:
test_preds_sk = dtc.predict_proba(sklearn_datadf_test.drop('y', axis=1))[:,1]

In [ ]:
fprs_sk, tprs_sk, _ = roc_curve(sklearn_datadf_test['y'], test_preds_sk)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(fprs_sk, tprs_sk, label= f'Sklearn implementations. AUC: {auc(fprs_sk, tprs_sk):0.3f}', color='g')
ax.plot(fprs, tprs, label= f'Scratch implementations. AUC: {auc_scratch(fprs, tprs):0.3f}', color='r')

ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
ax.set_title('ROC Curve')
ax.grid()
ax.legend()
fig.tight_layout()

Pretty close performance. Yay!!

# End